In [1]:
from imdb import IMDb
import pandas as pd
from pandas import DataFrame
import numpy as np
import re
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sw = stopwords.words('english')
from string import punctuation

In [2]:
df = pd.read_csv("DataSetShare.csv")
#df.head()

In [3]:
imdb = IMDb()

def get_imdb_info(title):
    runtime = []
    rating = []
    count = -1
    errors = 0
    identifier = []
    
    for i in title:
        count += 1
        # Search the first 30 characters on IMDb
        result = imdb.search_movie(i[:30])
        for i in range(len(result)):
            id = result[i].movieID
            identifier.append(id)
        try:
            mov = imdb.get_movie(result[0].movieID, info=['main'])
            runtime.append(int(mov.get('runtimes')[0]))
            rating.append(mov.get('rating'))
        except:
            runtime.append('')
            rating.append('')
            errors += 1
            # print(f'Error on index {count}, title: {i}') # Uncomment to investigate which titles were not found
    print(f'Total not found: {errors}')
    return pd.Series(runtime), pd.Series(rating), pd.Series(identifier)
    
runtime, rating, identifier = get_imdb_info(df.title)

df['runtime'] = runtime
df['rating'] = rating
df['ID'] = identifier

# Replace empty values with NaN
df = df.replace(r'^\s*$', np.NaN, regex=True)

df.head()


KeyboardInterrupt: 

This csv is what we will use for analytics. It is good to roll!

In [4]:
df.to_csv("IMDB_ratings.csv", index = False)


In [5]:
imdb = IMDb()
titles = []
ids = []
# Gets reviews for each stand up routine on IMDB
for item in df['ID']:
    title = imdb.get_movie_reviews(item)
    titles.append(title)
    ids.append(item)



This new DF is just to house the lists created from above, trying to break down the reviews column into individual reviews instead of a list of dictionaries

In [6]:
imdf = pd.DataFrame()
content =[]
helpful =[]
author = []
date_list =[]
rating = []
not_helpful = []
titl = []
ImID = []

for idx, reviews in enumerate(titles):
    this_id = ids[idx]
    if "reviews" not in reviews["data"]:
        continue
    for review_rec in reviews["data"]["reviews"]:
        content.append(review_rec["content"])
        helpful.append(review_rec["helpful"])
        titl.append(review_rec["title"])
        author.append(review_rec["author"])
        date_list.append(review_rec["date"])
        not_helpful.append(review_rec["not_helpful"])
        rating.append(review_rec["rating"])
        ImID.append(this_id)

imdf["ID"] = ImID
imdf["Reviews"] = content
imdf["Helpful"] = helpful
imdf["Review_Title"] = titl
imdf["Review_Author"] = author
imdf["Review_Date"] = date_list
imdf["Not_Helpful"] = not_helpful
imdf["Rating"] = rating



In [ ]:
reviews

In [ ]:
imdf.head()

In [7]:
tot_tok = []
unq_tok = []
avg_len = []
lex_div = []
tten = []
words = []
transcripts = []
for corpus in imdf["Reviews"]:
    

    total_tokens = 0
    unique_tokens = 0
    token_len = 0
    avg_token_len = 0
    lex_diversity = 0

    corpus = "".join([ch for ch in corpus if ch not in punctuation])
    includedTokens = [w for w in corpus.split()]
    fold = [w.lower() for w in includedTokens if w.isalpha()]
    fold  = [w for w in fold if w not in sw]
      
# Get the descriptive stats
    total_tokens = len(fold)

    unique_tokens = len(set(fold))
#     if total_tokens == 0:
#         break
    token_len = [len(w) for w in fold]
    avg_token_len = np.mean(token_len)

    lex_diversity = len(set(fold))/len(fold)

    fdtxt = FreqDist(fold)
    fdtxt = fdtxt.most_common(10)

    strtext = "".join([str(elem)for elem in fdtxt])
    top_10 = strtext

    tot_tok.append(total_tokens)
    unq_tok.append(unique_tokens)
    avg_len.append(avg_token_len)
    lex_div.append(lex_diversity)
    tten.append(top_10)
    words.append(fold)

In [ ]:
corpus

In [8]:
imdf["Total_Tokens"] = tot_tok
imdf["Unique_Tokens"] = unq_tok
imdf["Avg_Token_Length"] = avg_len
imdf["Lex_Diversity"] = lex_div
imdf["Most_Frequent_Words"] = tten
imdf["Words"] = words

In [9]:
imdf

,ID,Reviews,Helpful,Review_Title,Review_Author,Review_Date,Not_Helpful,Rating,Total_Tokens,Unique_Tokens,Avg_Token_Length,Lex_Diversity,Most_Frequent_Words,Words
0,6900644,Minhaj's recent turn at the Correspondent's Di...,0,,ur1002035,1 July 2017,0,NaN,130,108,6.300000,0.830769,"('show', 4)('special', 3)('much', 3)('performa...","[minhajs, recent, turn, correspondents, dinner..."
1,6900644,The guy works hard. He has an act close to Chr...,0,,ur18857804,28 July 2017,0,NaN,40,40,5.850000,1.000000,"('guy', 1)('works', 1)('hard', 1)('act', 1)('c...","[guy, works, hard, act, close, chris, rock, ev..."
2,6900644,6/5/18. Minhaj is funny. if you are looking fo...,0,,ur22652704,11 June 2018,0,NaN,12,11,5.750000,0.916667,"('minhaj', 2)('funny', 1)('looking', 1)('new',...","[minhaj, funny, looking, new, comedian, laugh,..."
3,6900644,"1 star might be a bit harsh, but, this had a 9...",0,,ur74642715,23 May 2017,0,NaN,157,131,5.515924,0.834395,"('bad', 4)('life', 4)('peters', 3)('sure', 2)(...","[star, might, bit, harsh, wrote, sure, netflix..."
4,6900644,This comedy special really just feels like you...,0,,ur71384678,24 October 2019,0,1.0,23,23,6.217391,1.000000,"('comedy', 1)('special', 1)('really', 1)('feel...","[comedy, special, really, feels, like, youre, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,5087554,I agree that dark comedy must not be taken ser...,0,,ur111645782,20 June 2021,0,NaN,17,17,5.823529,1.000000,"('agree', 1)('dark', 1)('comedy', 1)('must', 1...","[agree, dark, comedy, must, taken, serious, sh..."
2161,5087554,I would say the first 40 minutes is an ordinar...,0,,ur24103492,1 August 2021,0,NaN,63,55,6.714286,0.873016,"('even', 3)('minutes', 2)('deep', 2)('always',...","[would, say, first, minutes, ordinary, standup..."
2162,5087554,"This style of dark, edgy, quick fire jokes com...",0,,ur32900110,3 May 2019,0,NaN,16,16,5.500000,1.000000,"('style', 1)('dark', 1)('edgy', 1)('quick', 1)...","[style, dark, edgy, quick, fire, jokes, comedy..."
2163,5087554,I did not like this stand up comedy. I had to ...,0,,ur60771994,28 May 2018,0,NaN,90,74,5.888889,0.822222,"('comedy', 4)('jokes', 4)('child', 3)('stand',...","[like, stand, comedy, cut, child, molestation,..."


In [10]:
imdf.to_csv("IMDB_Review.csv")